In [ ]:
# 查看数据文件目录  list datalab files
!ls datalab/

In [ ]:
# 查看个人永久空间文件  list files in your permanent storage
!ls /home/tianchi/myspace/


In [ ]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

In [1]:
import torch
from torch import nn
import random
import numpy as np
from collections import defaultdict


In [2]:
class MY_RNN(nn.Module):
    def __init__(self,nwords,emb_size,hidden_size,ntags,layers):
        super(MY_RNN,self).__init__()
        self.emb=nn.Embedding(nwords,emb_size)
        self.rnn=nn.RNN(emb_size,hidden_size,layers)
        self.layer=nn.Linear(hidden_size,ntags)
    def forward(self,x):
        emb_out=self.emb(x)
        new_emb=emb_out.unsqueeze(1)
        out,_=self.rnn(new_emb)
        out=out[-1,:,:]
        layer_out=self.layer(out)
        return layer_out

In [3]:
w2i=defaultdict(lambda:len(w2i))
UNK=w2i["<unk>"]
t2i=defaultdict(lambda:len(t2i))

def read_dataset(filename):
    with open(filename,"r") as f:
        for line in f:
            tag,words=line.lower().strip().split(" ||| ")
            yield([w2i[word] for word in words.split(" ")],t2i[tag])

In [7]:
train=list(read_dataset("datalab/40056/train.txt"))
w2i=defaultdict(lambda :UNK,w2i)
dev=list(read_dataset("datalab/40056/dev.txt"))
nwords=len(w2i)
ntags=len(t2i)
print(nwords)
print(ntags)

EMB_SIZE=128
hidden_size=64
model=MY_RNN(nwords,EMB_SIZE,hidden_size,ntags,2)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

17614
5
MY_RNN(
  (emb): Embedding(17614, 128)
  (rnn): RNN(128, 64, num_layers=2)
  (layer): Linear(in_features=64, out_features=5, bias=True)
)


In [17]:
for epoch in range(10):
    random.shuffle(train)
    train_loss,train_correct=0.0,0
    print(len(train))
    for sentid,(words,tag) in enumerate(train):#遍历每一个训练样本
        input_words=torch.tensor(words)
        out_tag=torch.tensor([tag])
        out=model(input_words)
        loss=criterion(out,out_tag)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
        #print(out.shape)[1,5]
        predict = out[0].argmax().item()
        if predict == tag:
            train_correct += 1
        train_loss=loss.item()
        if(sentid+1%500==0):
            print("epoch=%r,sentid=%r,loss=%.4f,acc=%.3f"%(epoch,sentid,train_loss/len(sentid),train_correct/len(sentid)))
    print("epoch=%r,loss=%.4f,acc=%.3f"%(epoch,train_loss/len(train),train_correct/len(train)))

    model.eval()
    dev_correct=0
    for sentid,(words,tag) in enumerate(dev):#遍历每一个训练样本
        input_words=torch.tensor(words)
        out=model(input_words)
        predict = out[0].argmax().item()
        if predict == tag:
            dev_correct += 1
    print("epoch=%r,acc=%.3f"%(epoch,dev_correct/len(dev)))
            
        
        

8544
epoch=0,loss=0.0002,acc=0.283
epoch=0,acc=0.280
8544
epoch=1,loss=0.0002,acc=0.306
epoch=1,acc=0.282
8544
epoch=2,loss=0.0001,acc=0.320
epoch=2,acc=0.293
8544
epoch=3,loss=0.0002,acc=0.333
epoch=3,acc=0.247
8544
epoch=4,loss=0.0003,acc=0.356
epoch=4,acc=0.288
8544
epoch=5,loss=0.0002,acc=0.374
epoch=5,acc=0.281
8544
epoch=6,loss=0.0003,acc=0.390
epoch=6,acc=0.280
8544
epoch=7,loss=0.0001,acc=0.409
epoch=7,acc=0.298
8544
epoch=8,loss=0.0002,acc=0.428
epoch=8,acc=0.289
8544
epoch=9,loss=0.0003,acc=0.436
epoch=9,acc=0.278
